<a href="https://colab.research.google.com/github/brkived/Sure_PPO/blob/main/PPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/philtabor/Youtube-Code-Repository.git
%cd Youtube-Code-Repository/ReinforcementLearning/PolicyGradient/PPO/torch

Cloning into 'Youtube-Code-Repository'...
remote: Enumerating objects: 1044, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 1044 (delta 202), reused 187 (delta 183), pack-reused 824
Receiving objects: 100% (1044/1044), 43.11 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (587/587), done.
/content/Youtube-Code-Repository/ReinforcementLearning/PolicyGradient/PPO/torch


In [2]:
# This is the environment code for the Sure plus Summer 2024.
# Sara, Maitha, Batool, and Rand: Kindly note that I will update it reguraly, so
# whenever you run the simulation go and copy this code to your code
# Author: Dr. Abdulmalik Alwarafy, PhD. Summer 2024
#       *************************************************************
from gym import spaces
from gym.utils import seeding
import numpy as np # malik: a library for scientific computing
import cvxpy as cvx # maik library for optimization in python
import pandas as pd
#import colorama
#from colorama import Fore, Back, Style
import random
import copy

import os, time, sys # malik handels models savings
import numpy as np
import math
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib import mlab
import gym
from numpy import linalg as LA

# import Box2D  # maik I added this, I first "pip install box2d or box2d-py", then restarted kernel, related to LunarLanderContinuous-v2 env
# import winsound # for peep sound
# np.set_printoptions(precision=4)
# np.random.seed(0) # malik I added this
#!pip install box2d-py # a repackaged version of pybox2d

# malik at the beggining, make 2 directories "tmp/ddpg", manualy or typing followoing line in consol or in colab
# !mkdir tmp && mkdir tmp/ddpg   # creating directory for the models

#%% 1) Environment
class WirelessEnvironment:
    '''
The code should include 3 main compoenents:
1- Environment class: with intialization fucntion (of networks varialbles & state), step function (take action return s' & r)
2- Agent class: with memory function, DNN function, learn function: agent is single ground center GC
3- Main function: with main loopes and training

    '''

    def __init__(self, seed, No_TX_UAVs, No_Jam_UAVs, No_Eav):
        # ---------------------------------
        # 1) parameters related to UAVs, from the table in the paper
        # self.seed(seed=seed)
        self.set_seed(seed=seed)
        # np.random.seed(seed)
        self.No_TX_UAVs = No_TX_UAVs                   # number of transmit UAVs
        self.No_Jam_UAVs = No_Jam_UAVs                   # number of jamming UAVs
        self.No_Eav = No_Eav                   # number of ground eavesdroppers. I will consider them one
        self.Tot_No_UAVs = self.No_TX_UAVs + self.No_Jam_UAVs

        self.coverage_area = 200 # diameter in meter, not necessarly circular area. This x_min & y_min
        self.height_min, self.height_max = 20, 80 # heights of transmittng and jamming UAVs

        self.frequency= 2.4e9; # frequency of all RATs (in Hz)

        self.Wl_max = 20e6; # max (or available) BW of all UAVs (in Hz)
        # PSD or noise powers -174 dBm/Hz = 10^((-174-30)/10)
        T = 290 ; B = 15000 ; k_Boltzmann = 1.38e-23; self.PSD = k_Boltzmann*T*B*1e0 #* np.ones(shape=self.L)# (watts)
        self.exp_dist_mean = 2.46 # in my conference it was 2.46 dB mean of small-scale fading channel modeled as exponential RV (in dB)
        self.shad_fact = 1.8 # shadowing factor X of the PL (in dB)
        self.S_i = self.S_j = 0.5 # The harvesting efficiency of UAVs
        self.UAVs_speed = 10 #  np.random.randint(5, 20, self.Tot_No_UAVs)* (1000/3600) # speeds of UAVs (in m/s)
        self.Eav_speed = 2 # I assume eavesdropper speed is 2 m/s
        self.R_min_sec = 1e6 # bps  np.zeros(shape=(self.No_TX_UAVs, self.No_Eav), dtype=np.float32) # secrecy rate
        self.SNR_min_eav = 1.6 # dB
        self.B_max = 500# Jouls Maximum battery capacity of UAVs.
        UAVs_max_power_dBm = 30 ; self.P_max = 10**((UAVs_max_power_dBm - 30)/10) # max power in of trasmitting & Jamming UAVs in dBm
        self.d_min = 10 # m this is the in between distances between UAVs. I will consider transmitting UAVs
        self.pf = 1 # this is the flight power
        self.L = 10 # minimum distance between UAVs

        self.x_TX_UAVs = np.random.uniform(1, self.coverage_area, self.No_TX_UAVs) # Initial 3D locations of transmitting UAVs are random.
        self.y_TX_UAVs = np.random.uniform(1, self.coverage_area, self.No_TX_UAVs)
        self.z_TX_UAVs = np.random.uniform(self.height_min, self.height_max, self.No_TX_UAVs) # the z coordinates represents the height
        self.TX_UAVs_locations = np.stack([self.x_TX_UAVs, self.y_TX_UAVs, self.z_TX_UAVs], axis=1) # appending locations of trasmitting UAVs (to be optimized)

        self.x_Jam_UAVs = np.random.uniform(1, self.coverage_area, self.No_Jam_UAVs) # Initial 3D locations of jamming UAVs are random, we should optimize them
        self.y_Jam_UAVs = np.random.uniform(1, self.coverage_area, self.No_Jam_UAVs)
        self.z_Jam_UAVs = np.random.uniform(self.height_min, self.height_max, self.No_Jam_UAVs)
        self.Jam_UAVs_locations = np.stack([self.x_Jam_UAVs, self.y_Jam_UAVs, self.z_Jam_UAVs], axis=1) # appending locations of jamming UAVs (to be optimized)

        self.x_Eav = np.random.uniform(1, self.coverage_area, self.No_Eav) # Initial 3D locations of ground eavesdroppers are random (I assume one eav)
        self.y_Eav = np.random.uniform(1, self.coverage_area, self.No_Eav)
        self.z_Eav = 1.5# np.random.uniform(0, 1.5, self.No_Eav)
        self.Eav_locations = np.stack([self.x_Eav, self.y_Eav, [1.5] ], axis=1) # appending locations of eavesdropper

        self.Gr_cen_location = np.array([self.coverage_area/2, self.coverage_area/2, 1.5]) # locatoin of ground center is middle with height 1.5 m
        self.Char_sta_location = np.array([self.coverage_area/2, self.coverage_area/2, 3]) # locatoin of charging station is middle with height 3 m

        self.pi = np.ones(shape=(self.No_TX_UAVs, 1)) # vector of power allocatoin of transmitting UAVs. To be optimized
        self.pj = np.ones(shape=(self.No_Jam_UAVs, 1)) # vector of power allocatoin of jamming UAVs. To be optimized

        self.w = self.Wl_max/(self.No_TX_UAVs+self.No_Jam_UAVs) # bandwidth of all UAVs
        self.e_i_h = np.ones(shape=(self.No_TX_UAVs, 1)) # vector of harvested/transfered energy from charging station to transmitting UAVs. To be optimized
        self.e_j_h = np.ones(shape=(self.No_Jam_UAVs, 1)) # vector of harvested/transfered energy from charging station to transmitting UAVs. To be optimized
        self.b_i = np.ones(shape=(self.No_TX_UAVs, 1)) # energies intialization. Energy level
        self.b_j = np.ones(shape=(self.No_Jam_UAVs, 1)) # energies intialization
        self.E_i_h = np.ones(shape=(self.No_TX_UAVs, 1))
        self.E_j_h = np.ones(shape=(self.No_Jam_UAVs, 1))
        self.e_i_t = np.ones(shape=(self.No_TX_UAVs, 1)) # vector of transmitting energy of TX UAV
        self.e_j_t = np.ones(shape=(self.No_Jam_UAVs, 1)) # vector of transmitting energy of Jam UAV
        self.e_i_f = np.ones(shape=(self.No_TX_UAVs, 1)) # vector of flight energy of TX UAV
        self.e_j_f = np.ones(shape=(self.No_Jam_UAVs, 1)) # vector of flight energy of Jam UAV

        self.RiG = np.zeros(shape=(self.No_TX_UAVs, 1)) # legitimate rate
        self.Rie = np.zeros(shape=(self.No_TX_UAVs, self.No_Eav)) # illegitimate rate

        self.l = np.random.uniform(100, 200)*1e6 # transmitted packet size is from 100 to 200 Mbps. I assume Ii and Ij the same

        self.update_links()


    def calculate_distances(self):
      self.diG = LA.norm(self.TX_UAVs_locations - self.Gr_cen_location, axis=1) # distances between TX UAVs and ground center
      self.die = LA.norm(self.TX_UAVs_locations - self.Eav_locations, axis=1) # distances between TX UAVs and eavesdropper
      self.dje = LA.norm(self.Jam_UAVs_locations - self.Eav_locations, axis=1) # distances between jamming UAVs and eavesdropper
      self.dCi = LA.norm(self.Char_sta_location - self.TX_UAVs_locations, axis=1) # distances between charging station & TX UAVs
      self.dCj = LA.norm(self.Char_sta_location - self.Jam_UAVs_locations, axis=1) # distances between charging station & jamming UAVs
      a = np.zeros(shape=(self.No_TX_UAVs, self.No_Jam_UAVs)) # the dij matrix is NtxNj
      for j in range(self.No_Jam_UAVs):
        a[:, j] = LA.norm(self.TX_UAVs_locations - self.Jam_UAVs_locations[j], axis=1) # distances between TX UAVs & Jamming UAVs
      self.dij = a # LA.norm(self.TX_UAVs_locations - self.Jam_UAVs_locations, axis=1) # the dij matrix is NtxNj


    def calculate_gains_1(self):
      # generate channel gains: hiG (Ntx1), hie (NtxNe), hje (NjxNe)
      self.hiG = np.zeros(shape=(self.No_TX_UAVs, 1)) # channel between transmitting UAVs & ground station
      self.hie = np.zeros(shape=(self.No_TX_UAVs, self.No_Eav)) # channel between transmitting UAVs & eavesdropper
      self.hje = np.zeros(shape=(self.No_Jam_UAVs, self.No_Eav)) # channel between jamming UAVs & eavesdropper
      self.hCi = np.zeros(shape=(self.No_TX_UAVs, 1)) # channel between charging station & TX UAVs
      self.hCj = np.zeros(shape=(self.No_Jam_UAVs, 1)) # channel between charging station & jamming UAVs
      p_LoS_ij = PL_NLOS_ij = np.zeros(shape=(self.No_TX_UAVs, self.No_Jam_UAVs))
      self.hij = np.zeros(shape=(self.No_TX_UAVs, self.No_Jam_UAVs)) # channel between jamming UAVs &

      ALoS, ANLoS, reference_dist, beta, alpha = 10, 20, 1, 9.6, 0.28 #
      p_LoS_iG = 1/(1 + beta * np.exp(-alpha*(180/np.pi * np.arctan(self.TX_UAVs_locations[:, 2]/self.diG) - beta))) # probability of LoS. vector Nt
      p_LoS_ie = 1/(1 + beta * np.exp(-alpha*(180/np.pi * np.arctan(self.TX_UAVs_locations[:, 2]/self.die) - beta))) # probability of LoS. vector Nt
      p_LoS_je = 1/(1 + beta * np.exp(-alpha*(180/np.pi * np.arctan(self.Jam_UAVs_locations[:, 2]/self.dje) - beta))) # probability of LoS. vector Nj
      p_LoS_Ci = 1/(1 + beta * np.exp(-alpha*(180/np.pi * np.arctan(self.TX_UAVs_locations[:, 2]/self.dCi) - beta))) # probability of LoS. vector Nj
      p_LoS_Cj = 1/(1 + beta * np.exp(-alpha*(180/np.pi * np.arctan(self.Jam_UAVs_locations[:, 2]/self.dCj) - beta))) # probability of LoS. vector Nj
      a = np.zeros(shape=(self.No_TX_UAVs, self.No_Jam_UAVs)) # for the the p_LoS_ij matrix NtxNj
      for j in range(self.No_Jam_UAVs): # from jammer to TX UAVs
        a[:, j] = 1/(1 + beta * np.exp(-alpha*(180/np.pi * np.arctan(self.TX_UAVs_locations[:, 2]/self.dij[:, j]) - beta)))
      p_LoS_ij = a# 1/(1 + beta * np.exp(-alpha*(180/np.pi * np.arctan(self.TX_UAVs_locations[:, 2]/self.dij) - beta))) # probability of LoS. vector Nj

      PL_LOS_iG = ALoS + 20 * np.log10(4 * np.pi * self.frequency * self.diG/3e8) # LoS path-loss in dB. Vector size Nt
      PL_NLOS_iG = ANLoS + 20 * np.log10(4 * np.pi * self.frequency * self.diG/3e8) # NLoS path-loss in dB.
      PL_LOS_ie = ALoS + 20 * np.log10(4 * np.pi * self.frequency * self.die/3e8) # LoS path-loss in dB. Vector size Nt
      PL_NLOS_ie = ANLoS + 20 * np.log10(4 * np.pi * self.frequency * self.die/3e8) # NLoS path-loss in dB.
      PL_LOS_je = ALoS + 20 * np.log10(4 * np.pi * self.frequency * self.dje/3e8) # LoS path-loss in dB. Vector size Nt
      PL_NLOS_je = ANLoS + 20 * np.log10(4 * np.pi * self.frequency * self.dje/3e8) # NLoS path-loss in dB.
      PL_LOS_Ci = ALoS + 20 * np.log10(4 * np.pi * self.frequency * self.dCi/3e8) # LoS path-loss in dB. Vector size Nt
      PL_NLOS_Ci = ANLoS + 20 * np.log10(4 * np.pi * self.frequency * self.dCi/3e8) # NLoS path-loss in dB.
      PL_LOS_Cj = ALoS + 20 * np.log10(4 * np.pi * self.frequency * self.dCj/3e8) # LoS path-loss in dB. Vector size Nt
      PL_NLOS_Cj = ANLoS + 20 * np.log10(4 * np.pi * self.frequency * self.dCj/3e8) # NLoS path-loss in dB.
      PL_LOS_ij = ALoS + 20 * np.log10(4 * np.pi * self.frequency * self.dij/3e8) # LoS path-loss in dB. Vector size NtxNr
      PL_NLOS_ij = ANLoS + 20 * np.log10(4 * np.pi * self.frequency * self.dij/3e8) # NLoS path-loss in dB.

      hhiG = np.random.exponential(10**(self.exp_dist_mean/10), size = self.No_TX_UAVs) # small scale fading for the RF channel
      hhie = np.random.exponential(10**(self.exp_dist_mean/10), size = self.No_TX_UAVs)
      hhje = np.random.exponential(10**(self.exp_dist_mean/10), size = self.No_Jam_UAVs)
      hhCi = np.random.exponential(10**(self.exp_dist_mean/10), size = self.No_TX_UAVs)
      hhCj = np.random.exponential(10**(self.exp_dist_mean/10), size = self.No_Jam_UAVs)
      hhij = np.random.exponential(10**(self.exp_dist_mean/10), size = (self.No_TX_UAVs, self.No_Jam_UAVs))

      self.PL_iG = p_LoS_iG * PL_LOS_iG + (1 - p_LoS_iG) * PL_NLOS_iG # + np.random.normal(0, 10**(self.shad_fact/10), size=self.No_TX_UAVs) #path loss TX UAV & Ground BS # PL + shadowing in dB (0 mean & std =shad_fac)
      self.PL_ie = p_LoS_ie * PL_LOS_ie + (1 - p_LoS_ie) * PL_NLOS_ie # + np.random.normal(0, 10**(self.shad_fact/10), size=self.No_TX_UAVs)
      self.PL_je = p_LoS_je * PL_LOS_je + (1 - p_LoS_je) * PL_NLOS_je # + np.random.normal(0, 10**(self.shad_fact/10), size=self.No_Jam_UAVs)
      self.PL_Ci = p_LoS_Ci * PL_LOS_Ci + (1 - p_LoS_Ci) * PL_NLOS_Ci # + np.random.normal(0, 10**(self.shad_fact/10), size=self.No_TX_UAVs)
      self.PL_Cj = p_LoS_Cj * PL_LOS_Cj + (1 - p_LoS_Cj) * PL_NLOS_Cj # + np.random.normal(0, 10**(self.shad_fact/10), size=self.No_Jam_UAVs)
      self.PL_ij = p_LoS_ij * PL_LOS_ij + (1 - p_LoS_ij) * PL_NLOS_ij # + np.random.normal(0, 10**(self.shad_fact/10), size=(self.No_TX_UAVs, self.No_Jam_UAVs))

      self.hiG = np.power(10, -(self.PL_iG/10)) * (np.abs(hhiG))**2 # maybe with or without the second term (* (np.abs(hhiG))**2)
      self.hie = np.power(10, -(self.PL_ie/10)) * (np.abs(hhie))**2
      self.hje = np.power(10, -(self.PL_je/10)) * (np.abs(hhje))**2
      self.hCi = np.power(10, -(self.PL_Ci/10)) * (np.abs(hhCi))**2
      self.hCj = np.power(10, -(self.PL_Cj/10)) * (np.abs(hhCj))**2
      self.hij = np.power(10, -(self.PL_ij/10)) * (np.abs(hhij))**2
      print("*********hhiG", hhiG, "*******self.hiG", self.hiG)


    def calculate_gains(self):
      # generate channel gains: hiG (Ntx1), hie (NtxNe), hje (NjxNe)
      #self.hiG = np.zeros(shape=(self.No_TX_UAVs, 1)) # channel between transmitting UAVs & ground station
      #self.hie = np.zeros(shape=(self.No_TX_UAVs, self.No_Eav)) # channel between transmitting UAVs & eavesdropper
      #self.hje = np.zeros(shape=(self.No_Jam_UAVs, self.No_Eav)) # channel between jamming UAVs & eavesdropper
      #self.hCi = np.zeros(shape=(self.No_TX_UAVs, 1)) # channel between charging station & TX UAVs
      #self.hCj = np.zeros(shape=(self.No_Jam_UAVs, 1)) # channel between charging station & jamming UAVs
      #self.hij = np.zeros(shape=(self.No_TX_UAVs, self.No_Jam_UAVs)) # channel between jamming UAVs &

      PLE_LOS, reference_dist = 2, 1 # PL(dlu) = PLFS(d0)(dB) + 10nlog10( dlu/d0) +X(dB)
      FSPL = 20 * np.log10(4 * np.pi * self.frequency * reference_dist/3e8) # path-loss at a close-in reference distance d0
      hhiG = np.random.exponential(10**(self.exp_dist_mean/10), size = self.No_TX_UAVs) # small scale fading for the RF channel
      hhie = np.random.exponential(10**(self.exp_dist_mean/10), size = self.No_TX_UAVs)
      hhje = np.random.exponential(10**(self.exp_dist_mean/10), size = self.No_Jam_UAVs)
      hhCi = np.random.exponential(10**(self.exp_dist_mean/10), size = self.No_TX_UAVs)
      hhCj = np.random.exponential(10**(self.exp_dist_mean/10), size = self.No_Jam_UAVs)
      hhij = np.random.exponential(10**(self.exp_dist_mean/10), size = (self.No_TX_UAVs, self.No_Jam_UAVs)) # NtxNj
      self.PL_iG = FSPL + 10*(PLE_LOS) * np.log10(self.diG/reference_dist) + np.random.normal(0, 10**(self.shad_fact/10), size=self.No_TX_UAVs) # PL + shadowing in dB (0 mean & std =shad_fac)
      self.PL_ie = FSPL + 10*(PLE_LOS) * np.log10(self.die/reference_dist) + np.random.normal(0, 10**(self.shad_fact/10), size=self.No_TX_UAVs)
      self.PL_je = FSPL + 10*(PLE_LOS) * np.log10(self.dje/reference_dist) + np.random.normal(0, 10**(self.shad_fact/10), size=self.No_Jam_UAVs)
      self.PL_Ci = FSPL + 10*(PLE_LOS) * np.log10(self.dCi/reference_dist) + np.random.normal(0, 10**(self.shad_fact/10), size=self.No_TX_UAVs)
      self.PL_Cj = FSPL + 10*(PLE_LOS) * np.log10(self.dCj/reference_dist) + np.random.normal(0, 10**(self.shad_fact/10), size=self.No_Jam_UAVs)
      self.PL_ij = FSPL + 10*(PLE_LOS) * np.log10(self.dij/reference_dist) + np.random.normal(0, 10**(self.shad_fact/10), size=(self.No_TX_UAVs, self.No_Jam_UAVs))
      # print("mlaik check this ====>", hhiG,"------", self.PL_iG)

      self.hiG = np.power(10, -(self.PL_iG/10)) * (np.abs(hhiG))**2
      self.hie = np.power(10, -(self.PL_ie/10)) * (np.abs(hhie))**2
      self.hje = np.power(10, -(self.PL_je/10)) * (np.abs(hhje))**2
      self.hCi = np.power(10, -(self.PL_Ci/10)) * (np.abs(hhCi))**2
      self.hCj = np.power(10, -(self.PL_Cj/10)) * (np.abs(hhCj))**2
      self.hij = np.power(10, -(self.PL_ij/10)) * (np.abs(hhij))**2


    def calculate_SNRs(self):
      # calculate SNRs which need power, gains, BWs, and PSDs ==> gamma = h*p/(w*segma**2)
      self.gamma_iG = (self.hiG * np.reshape(self.pi, -1))/(self.w * self.PSD) # vector of length TX UAVs
      print("mlaik self.gamma_iG ====>", self.gamma_iG)
      self.gamma_ie = (self.hie * self.pi)/(self.w * self.PSD**2 + np.sum(self.hje * self.pj)) # from the conference paper
      a = np.zeros(shape=(self.No_TX_UAVs, self.No_Jam_UAVs)) # the dij matrix is NtxNj
      for j in range(self.No_Jam_UAVs): # from jammer to TX UAVs
        a[:, j] = (self.hij[:, j] * self.pj[j])/(self.w * self.PSD)
      self.gamma_ij = a

      self.I_tot = np.sum(np.log10(self.gamma_ij)) # the interference is assmed to be SNR


    def calculate_rates(self):
      # calculate the rates
      self.RiG = self.w * np.log2(1 + self.gamma_iG) # vector of length TX UAVs
      print("mlaik self.RiG ====>", self.RiG)
      self.Rie = self.w * np.log2(1 + self.gamma_ie) # vector of length TX UAVs
      self.Ri_sec = self.RiG - self.Rie # secrecy rate. vector of length TX UAVs
      print("mlaik self.RiG ====>", self.RiG, "---self.Rie=", self.Rie)
      self.Rsec_tot = np.sum(self.Ri_sec) # total secrecy rate of all TX UAVs


    def calculate_energies(self):
      #self.b_i = np.minimum( (self.b_i + self.E_i_h - self.e_i_t - self.e_i_f) , self.B_max)
      #self.b_j = np.minimum( (self.b_j + self.E_j_h - self.e_j_t - self.e_j_f) , self.B_max)
      self.e_i_t = (np.reshape(self.pi, -1) * self.l)/self.RiG # transmiting energy from TX UAVs. Vector of length Nt
      self.e_j_t = (np.reshape(self.pj, -1) * self.l)/np.sum(self.Rie) # transmiting energy from jamming UAVs. vector of length Nj. I have added the summation here

      self.e_i_f = (self.pf/self.UAVs_speed) * LA.norm(self.TX_UAVs_locations - self.TX_UAVs_locations, axis=1) # flight energy of TX UAVs. Vector of length Nt. To be updated
      self.e_j_f = (self.pf/self.UAVs_speed) * LA.norm(self.Jam_UAVs_locations - self.Jam_UAVs_locations, axis=1) # flight energy of jamming UAVs. Vector of length Nj

      self.e_i_h = np.ones(shape=(self.No_TX_UAVs, 1)) # harvested energy by TX UAVs. This is one of our actions. Vector Nt
      self.e_j_h = np.ones(shape=(self.No_Jam_UAVs, 1)) # harvested energy by Jam UAVs. This is one of our actions. Vector Nj

      self.E_i_h = np.reshape(self.hCi, -1) * self.S_i * np.reshape(self.e_i_h, -1) # Energy consumed by ith UAV harvesting technique. Vector Nt. gHr Si e^Hr_i ,
      self.E_j_h = self.hCj * self.S_j * np.reshape(self.e_j_h, -1) # Energy consumed by ith UAV harvesting technique. Vector Nt. gHr Si e^Hr_i ,

      self.b_i = np.minimum( (np.reshape(self.b_i, -1) + self.E_i_h - self.e_i_t - self.e_i_f) , self.B_max) # vector Nt
      self.b_j = np.minimum( (np.reshape(self.b_j, -1) + self.E_j_h - self.e_j_t - self.e_j_f) , self.B_max) # vector Nj

      self.E_tot = np.sum((self.e_i_t + self.e_i_f + self.e_i_h)/self.pi) + np.sum((self.e_j_t + self.e_j_f + self.e_j_h)/self.pj) # total energy effeciency


    def calculate_utility_function(self):
      # thi function calculates the objective function of our OPB U = w1 Rsec_tot + w2 E_tot + w3 I_tot
      self.w1, self.w2, self.w3 = 1,.1,.6# 8e-7, 1e-0, 1e-0
      #self.U = self.w1 * self.Rsec_tot + self.w2 * self.E_tot - self.w3 * self.I_tot
      #print("secrecy rate term utility=", self.w1 * self.Rsec_tot, "--- energy effeciency term utility=", self.w2 * self.E_tot,
      #      "--- interference term utility=", self.w3 * self.I_tot, "-----total utility", self.U)
      a1= np.sum(self.Ri_sec/np.max(self.Ri_sec));
      a2=(self.e_i_t + self.e_i_f + self.e_i_h)/self.pi ; a2 = np.sum(a2/np.max(a2))
      a3 =(self.e_j_t + self.e_j_f + self.e_j_h)/self.pj ; a3 = np.sum(a3/np.max(a3))
      a4=np.log10(self.gamma_ij) ; a4=np.sum(a4/np.max(a4))
      self.U = self.w1 * a1 + self.w2 * (a2+a3) - self.w3 * a4 # another version of utility
      print("secrecy rate term utility=", self.w1 * a1, "--- energy effeciency term utility=", self.w2 * (a2+a3),
            "--- interference term utility=", self.w3 * a4, "-----total utility", self.U)


    def update_links(self):
      # this function is called after each action at each time slot to regularly calculates the following parameters/functions:
      self.calculate_distances() # update distances based on the new UAV locations action
      # self.calculate_gains()     # update gains based on the new UAV distances
      self.calculate_gains_1()
      self.calculate_SNRs()      # update SNR based on the new UAV distances + power allocation action
      self.calculate_rates()     # update rates based on the new SNRs
      self.calculate_energies()  # update energies based on the new harvesting energy action
      self.calculate_utility_function()  # update total system utility


    def move_Eav_1(self): # malik I can use this for mobility model of eavesdropper, inspired by Faris
        # move eavesdropper randomly then I have to recalculate gains to all UAVs for each new EAV location
        theta = np.random.uniform(low=-np.pi, high=np.pi)
        dx = self.Eav_speed * np.cos(theta);
        dy = self.Eav_speed * np.sin(theta);
        self.Eav_locations[0] += dx
        self.Eav_locations[1] += dy
        # self.Eav_locations = np.stack([self.x_Eav, self.y_Eav, 1.5], axis=0) # get the new location of eavesdropper


    def move_Eav(self): # I may consider this
        """
        malik this is the Random Waypoint Mobility Model for an eavesdropper in a 3D space.
        time_step (float): The time step for each movement.
        pause_time (float): The time to pause at each waypoint.
        """
        pause_time = 0  # Start without pausing
        time_step = 1.0  # seconds

        if pause_time > 0: # If paused, return the old coordinates. I think I will delte this
            pause_time -= time_step
            return self.Eav_locations, pause_time

        # Select a new random waypoint
        new_x = np.random.uniform(0, self.converage_area) # xmin = 0, xmax= self.converage_area
        new_y = np.random.uniform(0, self.converage_area)
        # new_z = 1.5# np.random.uniform(zmin, zmax) # height of eavesdropper is 1.5m

        # Calculate the distance to the new waypoint
        distance = np.sqrt((new_x - self.Eav_locations[0])**2 + (new_y - self.Eav_locations[1])**2)
        travel_time = distance / self.Eav_speed # Calculate the time required to reach the new waypoint

        # Update coordinates based on speed and time_step
        if travel_time > time_step:
            ratio = time_step / travel_time
            new_x = self.Eav_locations[0] + (new_x - self.Eav_locations[0]) * ratio
            new_y = self.Eav_locations[1] + (new_y - self.Eav_locations[1]) * ratio
            # new_z = 1.5 # self.Eav_locations[2] + (new_z - self.Eav_locations[2]) * ratio
            pause_time = 0  # Not pausing yet
        else:
            pause_time = pause_time  # Pause at the new waypoint

        self.Eav_locations = np.stack([new_x, new_y, 1.5], axis=0)


    def set_seed(self, seed): # malik from there
        random.seed(seed) # Set the seed for random
        np.random.seed(seed) # Set the seed for numpy
        # Set the seed for torch
        T.manual_seed(seed)
        T.cuda.manual_seed(seed)
        T.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
        # Ensure reproducibility in torch.backends
        T.backends.cudnn.deterministic = True
        T.backends.cudnn.benchmark = False


    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]


    def reset_state(self):
      # S = [R_sec(t-1) (Ntx1), b(t-1) (Nt+Njx1), E^H(t-1)(Nt+Njx1), R^sec_min(1x1)] (3Nt + 2Nj +1)
      # np.random.seed(0) ; random.seed(0)
      a1 = (self.R_min_sec/self.No_TX_UAVs) * np.ones(self.No_TX_UAVs)
      a2 = np.ones(self.Tot_No_UAVs)
      a3 = np.ones(self.Tot_No_UAVs)
      a4 = np.array([self.R_min_sec])
      #self.state = np.concatenate( (a1, a2, a3, np.reshape(a4, (1,1)) ) , axis = 0) # malik this one was working
      self.state = np.concatenate( (a1, a2, a3, a4 ) , axis = 0)
      #self.state_spac = [ (self.R_min_sec/self.No_TX_UAVs) * np.ones(self.No_TX_UAVs), # R_sec(t-1) (Ntx1)
      #                    np.ones(self.Tot_No_UAVs), # b(t-1) (Nt+Njx1)
      #                    np.ones(self.Tot_No_UAVs), # E(t-1)(N_t+N_jx1)
      #                    np.reshape(np.array(self.R_min_sec), (1,1)) # R^sec_min(1x1)
      #                            ]
      #self.state = np.array(self.state_spac, dtype=object)
      #self.state = np.concatenate( (self.state_spac), axis = 0)
      # print(",,,,,,,,,,,,", np.shape(self.state))
      return self.state


    def check_constraints(self):  # malik I want to check the constraints
        self.Ri_sec_constraints = np.sum(self.Ri_sec > self.R_min_sec) == self.No_TX_UAVs # if this equals to 1, then C1 is met
        self.SNR_eav_constraints = np.sum(self.gamma_ie < self.SNR_min_eav) == self.No_TX_UAVs # if this equals to 1, then C2 is met

        a = np.zeros(shape=(self.No_TX_UAVs, self.No_TX_UAVs)) # distances between UAVs
        for i in range(self.No_TX_UAVs):
          for j in range(self.No_TX_UAVs):
            if i == j:
              a[i, j] = 0
            else:
              a[i, j] = LA.norm(self.TX_UAVs_locations[i] - self.TX_UAVs_locations[j], axis=0)
        self.in_between_distances = np.copy(a)
        self.d_min_constraints = np.sum(self.in_between_distances >= self.d_min) ==  (self.No_TX_UAVs*(self.No_TX_UAVs-1)) # if this equals to 1, then C7 is met

        self.max_travel_distance = LA.norm(self.TX_UAVs_locations - selold_TX_UAV_locatoins, axis=1) # current and past locatoins of TX UAVs. Ntx1
        # print("new locations:\n", self.TX_UAVs_locations, "old locations:\n", old_TX_UAV_locatoins)
        self.L_constraint = np.sum(self.max_travel_distance <= self.L) == self.No_TX_UAVs # if this equals to 1, then C6 is met

        print("Rate ConstraintC1:", self.Ri_sec_constraints, "SNR ConstraintC2:", self.SNR_eav_constraints, "dmin ConstraintC7:",
              self.d_min_constraints, "Lmin ConstraintC6:", self.L_constraint)
        self.costraints = self.Ri_sec_constraints & self.SNR_eav_constraints & self.d_min_constraints & self.L_constraint  # result is 1 or 0


    def DDPG_step(self, action): # our second main function
        self.reward_inst = 0.0 # instantionous reward
        self.done, self.abort = False, False

        state1 = self.state  # track old states if u might need them
        # S(t) = [R_sec(t-1) (Ntx1), b(t-1) (Nt+Njx1), E^H(t-1)(Nt+Njx1), R^sec_min(1x1)] (3Nt + 2Nj +1) ;
        R_sec = state1[0:self.No_TX_UAVs] # get current states
        b = state1[self.No_TX_UAVs : (2*self.No_TX_UAVs + self.No_Jam_UAVs)]
        E = state1[(2*self.No_TX_UAVs + self.No_Jam_UAVs) : (3*self.No_TX_UAVs + 2*self.No_Jam_UAVs)]
        R_sec_min = state1[(3*self.No_TX_UAVs + 2*self.No_Jam_UAVs) : (3*self.No_TX_UAVs + 2*self.No_Jam_UAVs+1)]# self.R_min_sec

        # A(t) = [Q (Nt+Nj x3), p (Nt+Nj x1), e^h (Nt+Nj x1)] 5(Nt+Nj)
        # the action should be a vector ==> [(Ntx3) + (Njx3) + (Nt) + (Nj) + (Nt) + (Nj)]
        # action = np.concatenate([a_TX_x, a_TX_y, a_TX_z, a_Jam_x, a_Jam_y, a_Jam_z, a_TX_p, a_Jam_p, a_TX_eh, a_Jam_eh], axis=0) # this is from main function
        TX_UAVs_locations = action[0 : 3*self.No_TX_UAVs] # get TX UAVs locations. Vector of length Ntx3
        Jam_UAVs_locations = action[3*self.No_TX_UAVs : 3*(self.No_TX_UAVs+self.No_Jam_UAVs)] # get Jam UAVs locations. Vector of length Njx3
        TX_UAVs_powers = action[3*(self.No_TX_UAVs+self.No_Jam_UAVs) : (4*self.No_TX_UAVs + 3*self.No_Jam_UAVs)] # get TX UAVs powers. Vector of length Nt
        Jam_UAVs_powers = action[(4*self.No_TX_UAVs + 3*self.No_Jam_UAVs) : 4*(self.No_TX_UAVs + self.No_Jam_UAVs)] # get Jam UAVs powers. Vector of length Nj
        TX_UAVs_harv_ener = action[4*(self.No_TX_UAVs + self.No_Jam_UAVs) : (5*self.No_TX_UAVs + 4*self.No_Jam_UAVs)] # get TX UAVs harvesting energies. Vector of length Nt
        Jam_UAVs_harv_ener = action[(5*self.No_TX_UAVs + 4*self.No_Jam_UAVs) : 5*(self.No_TX_UAVs + self.No_Jam_UAVs)] # get Jam UAVs harvesting energies. Vector of length Nj

        self.TX_UAVs_locations = np.reshape(TX_UAVs_locations, (self.No_TX_UAVs, 3), order='F') # convert vector into Ntx3 matrix
        # print("malik new locations===========>\n", self.TX_UAVs_locations)
        self.Jam_UAVs_locations = np.reshape(Jam_UAVs_locations, (self.No_Jam_UAVs, 3), order='F') # convert vector into Njx3 matrix
        self.pi = np.array(TX_UAVs_powers) # transmissoin power for trasmitting UAVs Nt
        self.pj = np.array(Jam_UAVs_powers) # transmissoin power for jamming UAVs Nj
        self.e_i_h = np.array(TX_UAVs_harv_ener) # harvested energy by TX UAVs. Vector Nt
        self.e_j_h = np.array(Jam_UAVs_harv_ener) # harvested energy by jam UAVs. Vector Nj

        self.update_links() # update all results/calculations based on the actions
        self.check_constraints() # check constraints
        # self.reward_inst += self.w1 * self.Rsec_tot + self.w2 * self.E_tot - self.w3 * self.I_tot # Initial: should be equal to the utiltiy function self.U
        self.reward_inst += self.U # reward is equal to the utiltiy function self.U

        if self.costraints:
          self.done = True
          self.abort = False
          # self.reward_inst += 0.0
        else:
          self.done = False
          self.abort = True
          # self.reward_inst += -1e2

          # now returen next state, reward, done, abort
          # S = [R_sec(t-1) (Ntx1), b(t-1) (Nt+Njx1), E^H(t-1)(Nt+Njx1), R^sec_min(1x1)] (3Nt + 2Nj +1)
          # print("mlaik see this ====>", np.shape(self.b_i), "mlaik see this ====>", np.shape(self.b_j))
          b = np.concatenate((self.b_i, self.b_j), axis = 0) # b(t-1) (Nt+Njx1)
          E = np.concatenate((self.E_i_h, self.E_j_h), axis = 0) # E(t-1) (Nt+Njx1)

          self.state = np.concatenate( (self.Ri_sec, b, E, np.reshape(self.R_min_sec, -1)) , axis = 0)

        return self.state, self.reward_inst, self.done#, self.abort_RATs


In [3]:
##ppo code

import os
import numpy as np
import torch as T
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical

class PPOMemory:
    def __init__(self, batch_size):
        self.states = []
        self.probs = []
        self.vals = []
        self.actions = []
        self.rewards = []
        self.dones = []

        self.batch_size = batch_size

    def generate_batches(self):
        n_states = len(self.states)
        batch_start = np.arange(0, n_states, self.batch_size)
        indices = np.arange(n_states, dtype=np.int64)
        np.random.shuffle(indices)
        batches = [indices[i:i+self.batch_size] for i in batch_start]

        return np.array(self.states),\
                np.array(self.actions),\
                np.array(self.probs),\
                np.array(self.vals),\
                np.array(self.rewards),\
                np.array(self.dones),\
                batches

    def store_memory(self, state, action, probs, vals, reward, done):
        self.states.append(state)
        self.actions.append(action)
        self.probs.append(probs)
        self.vals.append(vals)
        self.rewards.append(reward)
        self.dones.append(done)

    def clear_memory(self):
        self.states = []
        self.probs = []
        self.actions = []
        self.rewards = []
        self.dones = []
        self.vals = []

class ActorNetwork(nn.Module):
    def __init__(self, n_actions, input_dims, alpha,
            fc1_dims=256, fc2_dims=256, chkpt_dir='tmp/ppo'):
        super(ActorNetwork, self).__init__()

        self.checkpoint_file = os.path.join(chkpt_dir, 'actor_torch_ppo')
        self.actor = nn.Sequential(
                nn.Linear(*input_dims, fc1_dims),
                nn.ReLU(),
                nn.Linear(fc1_dims, fc2_dims),
                nn.ReLU(),
                nn.Linear(fc2_dims, n_actions),
                nn.Softmax(dim=-1)
        )

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        dist = self.actor(state)
        dist = Categorical(dist)

        return dist

    def save_checkpoint(self):
        T.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.checkpoint_file))

class CriticNetwork(nn.Module):
    def __init__(self, input_dims, alpha, fc1_dims=256, fc2_dims=256,
            chkpt_dir='tmp/ppo'):
        super(CriticNetwork, self).__init__()

        self.checkpoint_file = os.path.join(chkpt_dir, 'critic_torch_ppo')
        self.critic = nn.Sequential(
                nn.Linear(*input_dims, fc1_dims),
                nn.ReLU(),
                nn.Linear(fc1_dims, fc2_dims),
                nn.ReLU(),
                nn.Linear(fc2_dims, 1)
        )

        self.optimizer = optim.Adam(self.parameters(), lr=alpha)
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        value = self.critic(state)

        return value

    def save_checkpoint(self):
        T.save(self.state_dict(), self.checkpoint_file)

    def load_checkpoint(self):
        self.load_state_dict(T.load(self.checkpoint_file))

class Agent:
    def __init__(self, n_actions, input_dims, gamma=0.99, alpha=0.0003, gae_lambda=0.95,
            policy_clip=0.2, batch_size=64, n_epochs=10):
        self.gamma = gamma
        self.policy_clip = policy_clip
        self.n_epochs = n_epochs
        self.gae_lambda = gae_lambda

        self.actor = ActorNetwork(n_actions, input_dims, alpha)
        self.critic = CriticNetwork(input_dims, alpha)
        self.memory = PPOMemory(batch_size)

    def remember(self, state, action, probs, vals, reward, done):
        self.memory.store_memory(state, action, probs, vals, reward, done)

    def save_models(self):
        print('... saving models ...')
        self.actor.save_checkpoint()
        self.critic.save_checkpoint()

    def load_models(self):
        print('... loading models ...')
        self.actor.load_checkpoint()
        self.critic.load_checkpoint()

    def choose_action(self, observation):
        state = T.tensor([observation], dtype=T.float).to(self.actor.device)

        dist = self.actor(state)
        value = self.critic(state)
        action = dist.sample()

        probs = T.squeeze(dist.log_prob(action)).item()
        action = T.squeeze(action).item()
        value = T.squeeze(value).item()

        return action, probs, value

    def learn(self):
        for _ in range(self.n_epochs):
            state_arr, action_arr, old_prob_arr, vals_arr,\
            reward_arr, dones_arr, batches = \
                    self.memory.generate_batches()

            values = vals_arr
            advantage = np.zeros(len(reward_arr), dtype=np.float32)

            for t in range(len(reward_arr)-1):
                discount = 1
                a_t = 0
                for k in range(t, len(reward_arr)-1):
                    a_t += discount*(reward_arr[k] + self.gamma*values[k+1]*\
                            (1-int(dones_arr[k])) - values[k])
                    discount *= self.gamma*self.gae_lambda
                advantage[t] = a_t
            advantage = T.tensor(advantage).to(self.actor.device)

            values = T.tensor(values).to(self.actor.device)
            for batch in batches:
                states = T.tensor(state_arr[batch], dtype=T.float).to(self.actor.device)
                old_probs = T.tensor(old_prob_arr[batch]).to(self.actor.device)
                actions = T.tensor(action_arr[batch]).to(self.actor.device)

                dist = self.actor(states)
                critic_value = self.critic(states)

                critic_value = T.squeeze(critic_value)

                new_probs = dist.log_prob(actions)
                prob_ratio = new_probs.exp() / old_probs.exp()
                #prob_ratio = (new_probs - old_probs).exp()
                weighted_probs = advantage[batch] * prob_ratio
                weighted_clipped_probs = T.clamp(prob_ratio, 1-self.policy_clip,
                        1+self.policy_clip)*advantage[batch]
                actor_loss = -T.min(weighted_probs, weighted_clipped_probs).mean()

                returns = advantage[batch] + values[batch]
                critic_loss = (returns-critic_value)**2
                critic_loss = critic_loss.mean()

                total_loss = actor_loss + 0.5*critic_loss
                self.actor.optimizer.zero_grad()
                self.critic.optimizer.zero_grad()
                total_loss.backward()
                self.actor.optimizer.step()
                self.critic.optimizer.step()

        self.memory.clear_memory()


In [7]:
##main code

import gym
import numpy as np
from ppo_torch import Agent
from utils import plot_learning_curve

if __name__ == '__main__':
    seed = 100
    np.random.seed(seed)
    env = WirelessEnvironment(seed=seed, No_TX_UAVs=10, No_Jam_UAVs=2, No_Eav=1)
    N = 20
    max_step = 200
    normalize_state = 0
    evalute = False
    window = -100
    state_size = 3 * env.No_TX_UAVs + 2 * env.No_Jam_UAVs + 1
    action_size = 5 * (env.No_TX_UAVs + env.No_Jam_UAVs)
    batch_size = 5
    n_epochs = 4
    alpha = 0.0003
    agent = Agent(n_actions=action_size, batch_size=batch_size,
                  alpha=alpha, n_epochs=n_epochs,
                  input_dims=[state_size])
    n_games = 20


    figure_file = 'plots/cartpole.png'

    best_score = -1000000
    score_history = []

    learn_iters = 0
    avg_score = 0
    n_steps = 0

    old_TX_UAV_locations = np.copy(env.TX_UAVs_locations)

    for i in range(n_games):
        observation = env.reset_state()
        done = False
        score = 0
        while not done:
            action, prob, val = agent.choose_action(observation)
            observation_, reward, done, info = env.DDPG_step(action)
            n_steps += 1
            score += reward
            agent.remember(observation, action, prob, val, reward, done)
            if n_steps % N == 0:
                agent.learn()
                learn_iters += 1
            observation = observation_
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])

        if avg_score > best_score:
            best_score = avg_score
            agent.save_models()

        print('episode', i, 'score %.1f' % score, 'avg score %.1f' % avg_score,
                'time_steps', n_steps, 'learning_steps', learn_iters)
    x = [i+1 for i in range(len(score_history))]
    plot_learning_curve(x, score_history, figure_file)



*********hhiG [1.51556416 2.39039658 1.75272483 1.53626055 0.03638639 0.41539638
 1.38626201 2.58276839 0.50852298 0.59330356] *******self.hiG [2.35665697e-09 5.16919430e-09 3.48074324e-09 8.55919241e-10
 1.95013893e-13 3.62489919e-11 7.73095262e-10 3.00226264e-09
 1.76323774e-10 8.21743019e-10]
mlaik self.gamma_iG ====> [2.35547923e+01 5.16661099e+01 3.47900374e+01 8.55491495e+00
 1.94916434e-03 3.62308765e-01 7.72708908e+00 3.00076226e+01
 1.76235657e+00 8.21332353e+00]
mlaik self.RiG ====> [7.69655452e+06 9.53133833e+06 8.60247691e+06 5.42707172e+06
 4.68218786e+03 7.43422874e+05 5.20916754e+06 8.25758502e+06
 2.44316592e+06 5.33953613e+06]
mlaik self.RiG ====> [7.69655452e+06 9.53133833e+06 8.60247691e+06 5.42707172e+06
 4.68218786e+03 7.43422874e+05 5.20916754e+06 8.25758502e+06
 2.44316592e+06 5.33953613e+06] ---self.Rie= [[5.62050287e+05 4.80222246e+06 3.68538251e+06 2.02325987e+06
  9.17157810e+04 1.37049662e+04 2.70670841e+04 4.91226800e+05
  1.21629417e+04 1.41427243e+03]
 [5

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TypeError: 'int' object is not subscriptable

In [ ]:
##utils

import numpy as np
import matplotlib.pyplot as plt

def plot_learning_curve(x, scores, figure_file):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.title('Running average of previous 100 scores')
    plt.savefig(figure_file)